# Deep Finance Researcher with TODO Planner

Build an intelligent agent with task planning capabilities that can search financial documents (RAG) and live market data.

## Setup

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import create_agent
from langchain.agents.middleware import TodoListMiddleware, SummarizationMiddleware
from langgraph.checkpoint.sqlite import SqliteSaver
import sqlite3

from scripts.rag_tools import hybrid_search, live_finance_researcher
from scripts.prompts import MULTIMODEL_AGENT_PROMPT
from scripts.agent_utils import stream_agent_response

## Initialize Components

In [ ]:
model = ChatGoogleGenerativeAI(model='gemini-3-flash-preview')

## Create Agent with TODO Planner and Summarization

In [ ]:
def get_agent():

    conn = sqlite3.connect("data/todo_financial_rag_agent.db", check_same_thread=False)
    checkpointer = SqliteSaver(conn=conn)

    agent = create_agent(
        model=model,
        tools=[hybrid_search, live_finance_researcher],
        system_prompt=MULTIMODEL_AGENT_PROMPT,
        checkpointer=checkpointer,
        middleware=[
            SummarizationMiddleware(
                model=ChatGoogleGenerativeAI(model='gemini-3-flash-preview'),
                trigger=[("messages", 25)],
                keep=("messages", 10)
            ),
            TodoListMiddleware()
        ]
    )

    return agent

agent = get_agent()
agent

## Examples

In [ ]:
# Historical data query
stream_agent_response(agent, "What was Amazon's revenue in Q1 2024?", thread_id="session_1")

In [ ]:
# Live market data
stream_agent_response(agent, "What is the current stock price of Apple (AAPL) and latest news?", thread_id="session_2")

In [ ]:
# Complex query with TODO planning
stream_agent_response(
    agent, 
    "Compare Microsoft's Q2 2024 revenue from SEC filings with its current stock performance",
    thread_id="session_3"
)

In [ ]:
# Multi-company analysis
stream_agent_response(
    agent, 
    "Compare the profitability of Amazon and Google in 2023 and show results in a table",
    thread_id="session_4"
)